# Catastrophe

In [151]:
import warnings
warnings.filterwarnings('ignore')

In [316]:
# Define Neo4j connections
import pandas as pd
import random
from neo4j import GraphDatabase
host = 'bolt://localhost:7687'
user = 'neo4j'
password = 'password'
driver = GraphDatabase.driver(host,auth=(user, password))

# Configuration de l'option d'affichage
pd.set_option('display.max_rows', None)

In [317]:
def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [318]:
delete_node = """
    MATCH (n)
    WHERE NOT labels(n) = ['_GraphConfig']
    DETACH DELETE n
"""
run_query(delete_node)

""


In [319]:
deleConfig = """
CALL n10s.graphconfig.drop
"""
run_query(deleConfig)

n10s_config_query = """
CALL n10s.graphconfig.init({ handleVocabUris: "IGNORE", keepLangTag: false, handleMultival: "OVERWRITE"});
"""
run_query(n10s_config_query)

,param,value
0,handleVocabUris,IGNORE
1,handleMultival,OVERWRITE
2,handleRDFTypes,LABELS
3,keepLangTag,False
4,keepCustomDataTypes,False
5,applyNeo4jNaming,False
6,baseSchemaNamespace,neo4j://graph.schema#
7,baseSchemaPrefix,n4sch
8,classLabel,Class
9,subClassOfRel,SCO


In [320]:
n10s_config_query = """
CREATE CONSTRAINT  n10s_unique_uri IF NOT EXISTS FOR (r:Resource) REQUIRE r.uri IS UNIQUE;
"""
run_query(n10s_config_query)

""


In [321]:
import_ontology = """
    CALL n10s.onto.import.fetch("https://raw.githubusercontent.com/mouridoullah/elk-knowledge-graph/main/rdfData/file5.ttl", "Turtle") 
"""


run_query(import_ontology)

,terminationStatus,triplesLoaded,triplesParsed,namespaces,extraInfo,callParams
0,OK,229,231,None,,{'singleTx': False}


In [322]:
rsecouriste = """
CALL n10s.rdf.import.fetch("https://raw.githubusercontent.com/mouridoullah/elk-knowledge-graph/main/rdfData/file6.ttl","Turtle");
"""
run_query(rsecouriste)
labels = ["Médecin", "Militaire", "Infirmier", "Pompiers"]
for label in labels:
    query = f"MATCH (c:Resource {{name: '{label}'}}), (d:{label}) MERGE (d)-[:instanceOf]->(c)"
    run_query(query)

| Catastrophe  | QCode         | 
| :--------------- |:---------------:| 
|Séisme | Q7944 |
|Inondation | Q76337338|
|IncendieDeForet | Q169950|
|Tempete | Q81054|
|Secheresse | Q43059|
|GlissementDeTerrain | Q167903 |
|Tsunami | Q8070|
|cyclone | Q79602|
|AccidentDeTrain | Q1078765|
|IncendieDeMaison | Q7625093|
|Explosion | Q179057|
|AccidentNucleaire | Q1620824|

   

In [323]:
disasters = {
    "Seisme": "Q7944",
    "Inondation": "Q8068",
    "IncendieDeForet": "Q169950",
    "Tempete": "Q81054",
    "Secheresse": "Q43059",
    "Tsunami": "Q8070",
    "AccidentDeTrain": "Q1078765",
    "IncendieDeMaison": "Q7625093",
    "Explosion": "Q179057"
    # "AccidentNucleaire": "Q1620824",
    # "GlissementDeTerrain": "Q167903",
    # "Cyclone": "Q79602",
}

In [313]:
def get_random_disaster():
    liste_de_tuples = list(disasters.items())
    tuple_aléatoire = random.choice(liste_de_tuples)
    clé_aléatoire = tuple_aléatoire[0]
    valeur_aléatoire = tuple_aléatoire[1]
    return clé_aléatoire, valeur_aléatoire

In [314]:
clé_aléatoire, valeur_aléatoire = get_random_disaster()

query = f'''
WITH 
'
CONSTRUCT {{
  ?event a schema:{clé_aléatoire}, schema:Event ;
          schema:name ?eventLabel ;
          schema:description ?description ;
          schema:startDate ?date ;
          schema:aLieu ?location ;
          schema:numberOfInjured ?numberOfInjured ;
          schema:numberOfDeaths ?numberOfDeaths ;
          schema:aLieu ?locatedIn .
  ?location rdfs:label ?locationLabel ; a schema:Country, schema:SpatialThing.
  ?locatedIn rdfs:label ?locatedInLabel ; a schema:City, schema:SpatialThing.
}} WHERE {{
  ?event wdt:P31/wdt:P279* wd:{valeur_aléatoire} ;
          wdt:P17 ?location ;
          wdt:P585 ?date .
  OPTIONAL {{ ?event wdt:P2320 ?aftershocks . }}
  OPTIONAL {{ ?event wdt:P18 ?image . }}
  OPTIONAL {{ ?event wdt:P10 ?video . }}
  OPTIONAL {{ ?event wdt:P2528 ?magnitude . }}
  OPTIONAL {{ ?event wdt:P1339 ?numberOfInjured . }}
  OPTIONAL {{ ?event wdt:P1120 ?numberOfDeaths . }}
  OPTIONAL {{ ?event wdt:P131 ?locatedIn . ?locatedIn rdfs:label ?locatedInLabel . FILTER(LANG(?locatedInLabel) = "fr") }}
  OPTIONAL {{ ?event schema:description ?description . FILTER(LANG(?description) = "fr") }}
  ?location rdfs:label ?locationLabel . FILTER(LANG(?locationLabel) = "fr")
  ?event rdfs:label ?eventLabel . FILTER(LANG(?eventLabel) = "fr")
  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],fr". }}
}} ORDER BY RAND()
LIMIT 1
' AS sparql
CALL n10s.rdf.import.fetch(
  "https://query.wikidata.org/sparql?query=" + 
    apoc.text.urlencode(sparql),"JSON-LD", 
  {{ headerParams: {{ Accept: "application/ld+json"}} , 
    handleVocabUris: "IGNORE"}}) 
YIELD terminationStatus, triplesLoaded 
RETURN terminationStatus, triplesLoaded
'''

print(run_query(query), clé_aléatoire)

  terminationStatus  triplesLoaded
0                OK             14 IncendieDeForet


In [315]:
unwind = """
MATCH (n:Event)
WHERE n.uri IS NOT NULL
UNWIND n.startDate AS startDate
MERGE (n)-[:aPourDate]->(:Date { value: startDate })

WITH n
UNWIND n.numberOfInjured AS numberOfInjured
MERGE (n)-[:Implique]->(:Blesses { value: numberOfInjured })

WITH n
UNWIND n.numberOfDeaths AS numberOfDeaths
MERGE (n)-[:Implique]->(:Morts { value: numberOfDeaths })

WITH n
MATCH (n:Infirmier)
WHERE n.uri IS NOT NULL
UNWIND n.NombreDInfirmiers AS NombreDInfirmiers
MERGE (n)-[:related]->(:Int { value: NombreDInfirmiers })

RETURN n
"""
run_query(unwind)

,n


In [311]:
labels = ["Seisme", "Inondation", "IncendieDeForet", "Tempete", "Secheresse", "GlissementDeTerrain", "Tsunami", "Cyclone", "Date", "SpatialThing", 
          "AccidentDeTrain", "IncendieDeMaison", "Explosion", "AccidentNucleaire", "Morts", "Blesses"]


for label in labels:
    query = f"MATCH (c:Resource {{name: '{label}'}}), (d:{label}) MERGE (d)-[:instanceOf]->(c)"
    run_query(query)

In [264]:
# clé_aléatoire, valeur_aléatoire = get_random_disaster()

# query = f'''
# WITH 
# '
# CONSTRUCT {{
#   ?event a schema:{clé_aléatoire}, schema:Event ;
#           schema:name ?eventLabel ;
#           schema:description ?description ;
#           schema:startDate ?date ;
#           schema:aLieu ?location ;
#           schema:numberOfInjured ?numberOfInjured ;
#           schema:numberOfDeaths ?numberOfDeaths ;
#           schema:aLieu ?locatedIn .
#   ?location rdfs:label ?locationLabel ; a schema:Country, schema:SpatialThing.
#   ?locatedIn rdfs:label ?locatedInLabel ; a schema:City, schema:SpatialThing.
# }} WHERE {{
#   ?event wdt:P31/wdt:P279* wd:{valeur_aléatoire} ;
#           wdt:P17 ?location ;
#           wdt:P585 ?date .
#   OPTIONAL {{ ?event wdt:P2320 ?aftershocks . }}
#   OPTIONAL {{ ?event wdt:P18 ?image . }}
#   OPTIONAL {{ ?event wdt:P10 ?video . }}
#   OPTIONAL {{ ?event wdt:P2528 ?magnitude . }}
#   OPTIONAL {{ ?event wdt:P1339 ?numberOfInjured . }}
#   OPTIONAL {{ ?event wdt:P1120 ?numberOfDeaths . }}
#   OPTIONAL {{ ?event wdt:P131 ?locatedIn . ?locatedIn rdfs:label ?locatedInLabel . FILTER(LANG(?locatedInLabel) = "fr") }}
#   OPTIONAL {{ ?event schema:description ?description . FILTER(LANG(?description) = "fr") }}
#   ?location rdfs:label ?locationLabel . FILTER(LANG(?locationLabel) = "fr")
#   ?event rdfs:label ?eventLabel . FILTER(LANG(?eventLabel) = "fr")
#   SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],fr". }}
# }} LIMIT 5
# ' AS sparql
# CALL n10s.rdf.import.fetch(
#   "https://query.wikidata.org/sparql?query=" + 
#     apoc.text.urlencode(sparql),"JSON-LD", 
#   {{ headerParams: {{ Accept: "application/ld+json"}} , 
#     handleVocabUris: "IGNORE"}}) 
# YIELD terminationStatus, triplesLoaded 
# RETURN terminationStatus, triplesLoaded
# '''

# print(run_query(query), clé_aléatoire)


# unwind = """
# MATCH (n:Event)
# WHERE n.uri IS NOT NULL
# UNWIND n.startDate AS startDate
# MERGE (n)-[:aPourDate]->(:Date { value: startDate })

# WITH n
# UNWIND n.numberOfInjured AS numberOfInjured
# MERGE (n)-[:Implique]->(:Blesses { value: numberOfInjured })

# WITH n
# UNWIND n.numberOfDeaths AS numberOfDeaths
# MERGE (n)-[:Implique]->(:Morts { value: numberOfDeaths })

# WITH n
# MATCH (n:Infirmier)
# WHERE n.uri IS NOT NULL
# UNWIND n.NombreDInfirmiers AS NombreDInfirmiers
# MERGE (n)-[:related]->(:Int { value: NombreDInfirmiers })

# RETURN n
# """
# run_query(unwind)

# labels = ["Seisme", "Inondation", "IncendieDeForet", "Tempete", "Secheresse", "GlissementDeTerrain", "Tsunami", "Cyclone", "Date", "SpatialThing", 
#           "AccidentDeTrain", "IncendieDeMaison", "Explosion", "AccidentNucleaire", "Morts", "Blesses"]


# for label in labels:
#     query = f"MATCH (c:Resource {{name: '{label}'}}), (d:{label}) MERGE (d)-[:instanceOf]->(c)"
#     run_query(query)

  terminationStatus  triplesLoaded
0                OK             64 Explosion


In [251]:
# disasters = {
#     "Seisme": "Q7944",
#     "Inondation": "Q8068",
#     "IncendieDeForet": "Q169950",
#     "Tempete": "Q81054",
#     "Secheresse": "Q43059",
#     "GlissementDeTerrain": "Q167903",
#     "Tsunami": "Q8070",
#     "cyclone": "Q79602",
#     "AccidentDeTrain": "Q1078765",
#     "IncendieDeMaison": "Q7625093",
#     "Explosion": "Q179057",
#     "AccidentNucleaire": "Q1620824"
# }

# for disaster, qid in disasters.items():
#     query = f'''
#     WITH 
#     '
#     CONSTRUCT {{
#       ?event a schema:{disaster}, schema:Event ;
#              schema:name ?eventLabel ;
#              schema:description ?description ;
#              schema:startDate ?date ;
#              schema:location ?location ;
#              schema:image ?image ;
#              schema:video ?video ;
#              schema:hasSeverity ?magnitude ;
#              schema:numberOfInjured ?numberOfInjured ;
#              schema:numberOfDeaths ?numberOfDeaths ;
#              schema:hasAftershock ?aftershocks ;
#              schema:locatedInPlace ?locatedIn .
#       ?location rdfs:label ?locationLabel ; a schema:Country.
#       ?locatedIn rdfs:label ?locatedInLabel ; a schema:City.
#       ?date rdfs:label ?date ; a schema:date .
#     }} WHERE {{
#       ?event wdt:P31/wdt:P279* wd:{qid} ;
#              wdt:P17 ?location ;
#              wdt:P585 ?date .
#       OPTIONAL {{ ?event wdt:P2320 ?aftershocks . }}
#       OPTIONAL {{ ?event wdt:P18 ?image . }}
#       OPTIONAL {{ ?event wdt:P10 ?video . }}
#       OPTIONAL {{ ?event wdt:P2528 ?magnitude . }}
#       OPTIONAL {{ ?event wdt:P1339 ?numberOfInjured . }}
#       OPTIONAL {{ ?event wdt:P1120 ?numberOfDeaths . }}
#       OPTIONAL {{ ?event wdt:P131 ?locatedIn . ?locatedIn rdfs:label ?locatedInLabel . FILTER(LANG(?locatedInLabel) = "fr") }}
#       OPTIONAL {{ ?event schema:description ?description . FILTER(LANG(?description) = "fr") }}
#       ?location rdfs:label ?locationLabel . FILTER(LANG(?locationLabel) = "fr")
#       ?event rdfs:label ?eventLabel . FILTER(LANG(?eventLabel) = "fr")
#       SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],fr". }}
#     }} 
#     ' AS sparql
#     CALL n10s.rdf.import.fetch(
#       "https://query.wikidata.org/sparql?query=" + 
#         apoc.text.urlencode(sparql),"JSON-LD", 
#       {{ headerParams: {{ Accept: "application/ld+json"}} , 
#         handleVocabUris: "IGNORE"}}) 
#     YIELD terminationStatus, triplesLoaded 
#     RETURN terminationStatus, triplesLoaded
#     '''

#     print(run_query(query), disaster)
    

In [270]:
# # Liste des labels des catastrophes
# labels = ["Seisme", "Inondation", "IncendieDeForet", "Tempete", "Secheresse", "GlissementDeTerrain", "Tsunami", "cyclone", "AccidentDeTrain", "IncendieDeMaison", "Explosion", "AccidentNucleaire", "Pompiers", "Médecin", "Militaire", "Infirmier"]


# for label in labels:
#     # Requête Cypher pour créer la relation "instanceOf"
#     query = f"MATCH (c:Class {{name: '{label}'}}), (d:{label}) MERGE (d)-[:instanceOf]->(c)"
#     # Exécution de la requête
#     run_query(query)

In [91]:
seism_count_query = """
    MATCH (n:Seisme)
    RETURN count(*)
"""

run_query(seism_count_query)

,count(*)
0,0


In [92]:
disasterTurquie = """
    MATCH (n:Country)-[]-(related)
    WHERE any(label IN n.label WHERE label CONTAINS 'Turquie') OR any(label IN n.label WHERE label CONTAINS 'Syrie')
    RETURN n.label, related.name, related.description, related.startDate
    ORDER BY related.startDate DESC
"""

run_query(disasterTurquie)

,n.label,related.name,related.description,related.startDate


In [ ]:
inferCatastropheNaturelle = """
  CALL n10s.inference.nodesLabelled('CatastropheNaturelle',  {
    catNameProp: "name",
    catLabel: "Resource",
    subCatRel: "SCO"
  })
  YIELD node
  RETURN labels(node), node.name, node.description
  ORDER BY labels(node)
"""

run_query(inferCatastropheNaturelle)

In [ ]:
user = """
    MERGE (jb:User { userId : "JB2020"}) with jb
    MATCH (e1:Event { uri: "http://www.wikidata.org/entity/Q192050" })
    MATCH (e2:Event { uri: "http://www.wikidata.org/entity/Q1808853" })
    WITH jb, e1, e2
    MERGE (e1)<-[:INTERESTED_IN]-(jb)-[:INTERESTED_IN]->(e2)
"""

run_query(user)

In [ ]:
query = """
    MATCH (:User { userId : "JB2020"})-[:INTERESTED_IN]->(b:Event)
    WHERE n10s.inference.hasLabel(b,'Catastrophe',({ catNameProp: "name", catLabel: "Resource", subCatRel: "SCO" }))
    RETURN b.uri as uri, b.name as name, labels(b) as categories;
"""

run_query(query)

In [ ]:
infer = """
    MATCH (cat:Resource { name: "CatastropheTechnologique"})
    CALL n10s.inference.nodesInCategory(cat, { inCatRel: "instanceOf", subCatRel: "SCO"}) yield node
    RETURN labels(node), node.name, node.description
    ORDER BY labels(node)
"""

run_query(infer)

In [ ]:
delete_node = """
    MATCH (n)
    WHERE NOT labels(n) = ['_GraphConfig']
    DETACH DELETE n
"""
run_query(delete_node)

In [ ]:
data = """
CALL n10s.onto.import.fetch("https://github.com/neo4j-labs/neosemantics/raw/3.5/docs/rdf/movieDBRelHierarchy.ttl", "Turtle");
"""
run_query(data)

In [ ]:
infer = """
    match (thematrix:Movie {title: "The Matrix"})
    call n10s.inference.getRels(thematrix,"WORKED_IN", { subRelRel: "SPO" }) yield rel, node
    return type(rel) as relType, node.name, node.born
"""

run_query(infer)

In [ ]:
# unwind = """
# MATCH (n:Resource)
# WHERE n.name IS NOT NULL
# WITH n
# UNWIND n.hasSeverity AS severity
# CREATE (n)-[:HAS_SEVERITY]->(:Severity {value: severity})
# WITH n
# UNWIND n.startDate AS date
# CREATE (n)-[:START_DATE]->(:StartDate {value: date})
# WITH n
# UNWIND n.numberOfInjured AS injured
# CREATE (n)-[:NUMBER_OF_INJURED]->(:NumberOfInjured {value: injured})
# WITH n
# UNWIND n.numberOfDeaths AS deaths
# CREATE (n)-[:NUMBER_OF_DEATHS]->(:NumberOfDeaths {value: deaths})
# WITH n
# UNWIND n.hasAftershock AS aftershock
# CREATE (n)-[:HAS_AFTERSHOCK]->(:Aftershock {value: aftershock})

# RETURN n
# """
# run_query(unwind)

In [ ]:
# import_ontology = """
# CALL n10s.onto.import.fetch("https://raw.githubusercontent.com/mouridoullah/elk-knowledge-graph/main/rdfData/cata.ttl", "Turtle") 
# """
# run_query(import_ontology)

# import_data = """
# CALL n10s.rdf.import.fetch("https://raw.githubusercontent.com/mouridoullah/elk-knowledge-graph/main/rdfData/data.ttl","Turtle");
# """

# run_query(import_data)

In [ ]:
# onto = """ 
# CREATE (cata:Class { authoritativeLabel: "Catastrophe", name: "Catastrophe", identifier: "sh 8503449854" })
# CREATE (c:Class { authoritativeLabel: "Catastrophe Naturelle", name: "CatastropheNaturelle", identifier: "sh 85034498" })
# CREATE (po:Class { authoritativeLabel: "Explosion", name: "Explosion", identifier: "sh 85095187" })
# CREATE (s:Class { authoritativeLabel: "Incendie De Foret", name: "IncendieDeForet", identifier: "sh 85124647" })


# CREATE (c)<-[:SCO]-(:Class { authoritativeLabel: "Cyclone", name: "Cyclone", identifier: "sh 85034488" })-[:SCO]->(po)
# CREATE (c)<-[:SCO]-(:Class { authoritativeLabel: "Tsunami", name: "Tsunami", identifier: "sh 85034503" })-[:SCO]->(s)
# CREATE (c)<-[:SCO]-(:Class { authoritativeLabel: "Tempete", name: "Tempete", identifier: "sh 2007001101" })
# CREATE (c)<-[:SCO]-(:Class { authoritativeLabel: "Seisme", name: "Seisme", identifier: "sh 85068653" })
# CREATE (cata)<-[:SCO]-(c)
# """

# run_query(onto)

In [ ]:
# data = """
# CREATE (:Event:Cyclone { title: "Cyclone", identifier: "2167673"})
# CREATE (:Event:Tempete { title: "Tempete", identifier: "11916857"})

# CREATE (:Event:Seisme { title: "Seisme 1", identifier: "8096161"})

# CREATE (:Event:Seisme { title: "Seisme 2", identifier: "12873809"})
# CREATE (:Event:Tsunami { title: "Tsunami", identifier: "20234576"})
# """

# run_query(data)

In [ ]:
# infer = """CALL n10s.inference.nodesLabelled('CatastropheNaturelle',  {
#   catNameProp: "name",
#   catLabel: "Class",
#   subCatRel: "SCO"
# })
# YIELD node
# RETURN labels(node), node.name, node.description
# """

# run_query(infer)

In [21]:
from SPARQLWrapper import SPARQLWrapper, JSON

# Endpoint Wikidata
endpoint_url = "https://query.wikidata.org/sparql"

# Requête pour récupérer les sous-classes de chaque classe
query_template = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT ?subclass ?label WHERE {{
  ?subclass rdfs:subClassOf wd:{class_id}.
  ?subclass rdfs:label ?label .
  FILTER(LANG(?label) = "fr")
}} 
"""

# Fonction pour exécuter la requête pour chaque classe
def get_subclasses(class_id):
    sparql = SPARQLWrapper(endpoint_url)
    query = query_template.format(class_id=class_id)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    subclasses = [(result["subclass"]["value"], result["label"]["value"]) for result in results["results"]["bindings"]]
    return subclasses

# Exemple d'utilisation
class_id = "Q3839081"  # Id de la classe "catastrophe" dans Wikidata
subclasses = get_subclasses(class_id)
print(subclasses)


[]
